In [1]:
import numpy as np
from numpy import dot
from numpy.linalg import norm
import pandas as pd
from tqdm import tqdm
from itertools import product

In [2]:
NUM_USERS = 53897
NUM_ITEMSETS = 27694
NUM_ITEMS = 42653

In [3]:
train_df = pd.read_csv('/home/nas3_userL/sohyunjeong/work_dir/etc/23spring/hw2/dataset/itemset_item_training.csv', names=['itemset_id', 'item_id'])
valid_df = pd.read_csv('/home/nas3_userL/sohyunjeong/work_dir/etc/23spring/hw2/dataset/itemset_item_valid_query.csv', names=['itemset_id', 'item_id'])
valid_ans = pd.read_csv('/home/nas3_userL/sohyunjeong/work_dir/etc/23spring/hw2/dataset/itemset_item_valid_answer.csv', names=['itemset_id', 'answer'])
test_df = pd.read_csv('/home/nas3_userL/sohyunjeong/work_dir/etc/23spring/hw2/dataset/itemset_item_test_query.csv', names=['itemset_id', 'item_id'])

In [4]:
train_itemsets = list(set(train_df['itemset_id'].tolist()))
train_dict = dict()
for train_itemset in tqdm(train_itemsets):
    items = train_df[train_df['itemset_id']==train_itemset]['item_id'].tolist()
    train_dict.update({train_itemset:items})
    
train_itemset_item = np.zeros((len(train_dict.keys()), NUM_ITEMS+1))
for i, (k, v) in enumerate(train_dict.items()) :
    train_itemset_item[i,NUM_ITEMS] = k
    train_itemset_item[i,v] = 1

100%|██████████| 22156/22156 [00:04<00:00, 5229.70it/s]


In [5]:
train_itemset_item.shape

(22156, 42654)

In [6]:
valid_itemsets = list(set(valid_df['itemset_id'].tolist()))
valid_dict = dict()
for valid_itemset in tqdm(valid_itemsets):
    items = valid_df[valid_df['itemset_id']==valid_itemset]['item_id'].tolist()
    valid_dict.update({valid_itemset:items})

valid_itemset_item = np.zeros((len(valid_dict.keys()), NUM_ITEMS+1))
for i, (k, v) in enumerate(valid_dict.items()) :
    valid_itemset_item[i,NUM_ITEMS] = k
    valid_itemset_item[i,v] = 1

100%|██████████| 2769/2769 [00:00<00:00, 5665.36it/s]


In [7]:
test_itemsets = list(set(test_df['itemset_id'].tolist()))
test_dict = dict()
for test_itemset in tqdm(test_itemsets):
    items = test_df[test_df['itemset_id']==test_itemset]['item_id'].tolist()
    test_dict.update({test_itemset:items})
    
test_itemset_item = np.zeros((len(test_dict.keys()), NUM_ITEMS+1))
for i, (k, v) in enumerate(test_dict.items()) :
    test_itemset_item[i,NUM_ITEMS] = k
    test_itemset_item[i,v] = 1

100%|██████████| 2769/2769 [00:00<00:00, 5983.95it/s]


In [8]:
itemset_item = np.concatenate((train_itemset_item, valid_itemset_item, test_itemset_item), axis=0)
mul_itemset_item = itemset_item[:,:-1]@itemset_item[:,:-1].T
norm = np.sqrt(np.diag(mul_itemset_item))[:,None]@(np.sqrt(np.diag(mul_itemset_item))[:,None]).T
itemset_sim = np.divide(mul_itemset_item, norm)
itemset_sim = itemset_sim*(1-np.eye(itemset_sim.shape[0],itemset_sim.shape[0]))

In [9]:
results = np.dot(itemset_sim, itemset_item[:,:-1])[22156:22156+2769]
ranking = np.argsort(results, 1)[::-1][:,:100]

In [10]:
acc = []
rank = []

for i , itemset_id in enumerate(tqdm(valid_itemset_item[:,-1])):
    ans = valid_ans[valid_ans['itemset_id']==itemset_id]['answer'].tolist()[0]
    if ans in ranking[i]:
        acc.append(1)
        rank.append(np.where(ranking[i]==ans)[0][0])
    else:
        acc.append(0)
        rank.append(101)

100%|██████████| 2769/2769 [00:00<00:00, 5534.91it/s]


In [11]:
print(np.array(acc).sum()/len(acc))
print(np.array(rank).sum()/len(acc))

0.0021668472372697724
100.89888046226075
